In [1]:
import numpy as np
import pandas as pd
import wrangle_anomaly as w

import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
traffic = w.wrangle_web_traffic()

opening file from local directory


AttributeError: 'Series' object has no attribute 'path'

In [ ]:
traffic

# 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?

In [ ]:
traffic.program_id.value_counts(dropna=False)

In [ ]:
traffic.reset_index().groupby(['lesson', 'program_id']).\
    datetime.nunique().sort_values(ascending=False).head(50)

In [ ]:
traffic[traffic.program_id == 4].reset_index().groupby(['lesson', 'program_id']).\
    datetime.nunique().sort_values(ascending=False).head(50)

In [ ]:
traffic[traffic.program_id.isna() == True].reset_index().groupby(['lesson']).\
    datetime.nunique().sort_values(ascending=False).head(50)

### The most commonly accessed lesson for each program were:
- Program 1 - 'content' which I don't think is a lesson, 'javascript-i' was the next highest
- Program 2 - 'javascript-i'
- Program 3 - 'sql'
- Program 4 - 'content'

# 2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

In [ ]:
lesson_df1 = pd.DataFrame(traffic[traffic.cohort_name != 'Staff'].\
                          reset_index().groupby('lesson').\
     cohort_name.value_counts()).rename(columns={'cohort_name':'count'}).\
    reset_index()

In [ ]:
lesson_df1.head()

In [ ]:
lesson_df2 = pd.DataFrame(traffic[traffic.cohort_name != 'Staff']\
                          .reset_index().groupby('lesson').\
     cohort_name.value_counts(normalize=True)).rename(columns={'cohort_name':'proba'}).\
    reset_index()

In [ ]:
lesson_df2.head()

In [ ]:
# merge the two data frames create above into a single one:
lesson_df = pd.concat([lesson_df1, lesson_df2.proba], axis=1)
lesson_df.head()

In [ ]:
lesson_df[(lesson_df['count'] >= 10)].\
    sort_values('proba', ascending=False).head(50)

# 3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?

In [ ]:
traffic.head(2)

#### To get accesses during active status, we will need to find all instances where datetimes is between start_date and end_date for each user

In [ ]:
active_df = traffic[(traffic.start_date < traffic.index)
                    & (traffic.end_date > traffic.index)
                    & (traffic.cohort_name != 'Staff')]

In [ ]:
active_df.groupby('user_id').lesson.count().sort_values().head(50)

In [ ]:
active_df.groupby('user_id').lesson.count().sort_values().head(50).index

In [ ]:
low_activity_users = active_df.groupby('user_id').\
    lesson.count().sort_values().head(50).index

In [ ]:
low_activity_df = pd.DataFrame(active_df[active_df.user_id.isin(low_activity_users)])

In [ ]:
low_activity_user_df = pd.DataFrame(active_df[active_df.user_id.isin(low_activity_users)]\
    [['user_id','ip','cohort_name','start_date', 'end_date']].\
    groupby('user_id').value_counts()).rename(columns={0:'counts'}).reset_index()

In [ ]:
low_activity_user_df

In [ ]:
low_activity_df.head()

In [ ]:
low_activity_df.groupby('cohort_name').user_id.nunique()

#### There are 49 users that accessed the content less than 70 times while active. The most users with low access came from cohorts Oberon and Sequoia

# 4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?

In [ ]:
traffic[traffic.cohort_id.isna() == True]

### 5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?

In [ ]:
traffic.lesson.value_counts().head(20)

In [ ]:
traffic[traffic.lesson == 'javascript-i'].groupby('program_id').\
    resample('D').user_id.nunique().reset_index()

In [ ]:
js_1 = traffic[traffic.lesson == 'javascript-i'].groupby('program_id').\
    resample('D').user_id.nunique().reset_index()

In [ ]:
js_1 = traffic[(traffic.lesson == 'classification') 
               &(traffic.program_id == 2)
              &(traffic.index > '2019-09')].groupby('program_id').\
    resample('D').user_id.nunique().reset_index()

In [ ]:
sns.lineplot(data=js_1, x='datetime', y='user_id', hue='program_id', alpha=0.5)
plt.xticks(rotation=90)
plt.show()

- The last access to Javascript lessons for program 1 was on 2020-01-15.
- program 2 appears to be able to access all content
- The last access to Javascript lessons for program 3 was on 2019-12-15.

# 6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?

In [ ]:
after_grad = traffic[(traffic.index > traffic.end_date) & (traffic.cohort_name != 'Staff')]

In [ ]:
after_grad.head(2)

In [ ]:
after_grad.groupby('program_id','lesson').index.nunique()

# 7. Which lessons are least accessed?